In [1]:
from TimeTagger import FileReader
import numpy as np
import easygui

We know that H and V got swapped in BOB so I must correct for it in code. 

In [2]:
#GLOBAL DEFINITIONS
H = 1
V = 2
A = 3
D = 4


In [3]:
#folder = r'C:\Data\\'
folder = "./swabian/20200714_Swabian_Timestamp/data_2"
#file = '2.97mW_1.1.ttbin'
file = "4_1uW0dB.1.ttbin"
filepath = folder + '/' + file

#filepath = easygui.fileopenbox(default = folder)






In [4]:
def loadSwabianTimeStamps(fileIn, t = np.inf):
    
        file_reader = FileReader(fileIn)
        stamps = []
        channel = []
        
        
        while file_reader.hasData():
            #n_events = 100000; # Number of events to read at once
            n_events = 200000; # Number of events to read at once
            
            # Read at most n_events.
            data = file_reader.getData(n_events);  # data is an instance of TimeTagStreamBuffer

            channelTmp = data.getChannels()
            stampsTmp = data.getTimestamps()
          #  overflow_types = data.getEventTypes()   # TimeTag = 0, Error = 1, OverflowBegin = 2, OverflowEnd = 3, MissedEvents = 4
          #  missed_events = data.getMissedEvents()
            
            for i in range(len(channelTmp)):
                channel.append(channelTmp[i])
                stamps.append(stampsTmp[i])
            break
            if stamps[-1] - stamps[0] > t:
                break
        return stamps, channel


In [5]:
stamps, channel = loadSwabianTimeStamps(filepath)

In [6]:
channel[:20]

[2, 6, 2, 8, 1, 2, 4, 4, 5, 8, 5, 7, 3, 7, 4, 6, 5, 6, 1, 5]

In [7]:
#seperate out alice's and bob's events
astamps = []
achannel = []
bstamps = []
bchannel = []
for (s,c) in zip(stamps, channel):
    if (c <=4):
        astamps.append(s)
        achannel.append(c)
    else:
        bstamps.append(s)
        nc = c-4
        
        if (nc==3):
            nc = 4
        elif (nc == 4):
            nc = 3
        bchannel.append(nc)

In [8]:
len(astamps)

89959

In [126]:
len(bstamps)

110041

In [127]:
577447917

577447917

In [128]:
len(bstamps)+len(astamps)

200000

In [114]:
#convert the resolution to 1/8 of a nanosecond
saverawevents

NameError: name 'saverawevents' is not defined

In [115]:
astamps[-1]

1101535893176

In [116]:
achannel[-1]

2

In [117]:
astamps[:10]

[1086398751803,
 1086398894346,
 1086398985766,
 1086399116932,
 1086399157249,
 1086399262520,
 1086399490280,
 1086399689867,
 1086400241735,
 1086400301477]

In [118]:
achannel[:10]

[2, 2, 1, 2, 4, 4, 3, 4, 1, 2]

In [119]:
len(astamps)

67356

In [120]:
2197945792/64

34342903.0

In [129]:
def coincount(at,ad,bt,bd,amd=H,bmd=H,cwindow=1000,shift=0 ):
    #print("coincount: ",amd,bmd,cwindow,shift)
    ia = 0
    ib = 0
    coin_count = 0
    while (ia<len(at) and ib <len(bt)):
        #ignore beacon events
        if (ad[ia]!=amd):
            ia+=1
            continue
        if (bd[ib]!=bmd):
            ib+=1
            continue
            
        diff = bt[ib]- at[ia] + shift

        if (abs(diff)<=cwindow):
            coin_count +=1
            
            ia+=1
            ib+=1
            #continue
        elif (diff>cwindow):
            ia+=1
            #continue
        elif( diff <-cwindow):
            ib+=1
            #continue
        else:
            print ("ERROR! I should not be here")
            break
    return coin_count


In [70]:
#x = coincount(astamps, achannel,bstamps, bchannel, H, H,cwindow=1000,shift=0)
#x

In [130]:
def g2_heisto(at,ad,bt,bd,amd=H,bmd=H,cwindow=1000,search_steps=10,search_direction=1):
    heisto = []
    i = 0
    count = 0
    while(i<search_steps):
        shift = i*cwindow*search_direction
        count = coincount(at, ad,bt, bd, amd,bmd,cwindow=cwindow,shift=shift)
        #print (count)
        heisto.append(count)
        i += 1
    return heisto 

In [134]:
h = g2_heisto(astamps, achannel,bstamps, bchannel, amd=H ,bmd=A,cwindow=500,search_steps=20,search_direction=-1)
print (h)
h.index(max(h))

[35, 30, 27, 33, 32, 28, 33, 39, 30, 35, 48, 55, 79, 307, 1040, 1093, 361, 74, 45, 48]


15

In [132]:
detectors = ["_","H","V","A","D"]

In [133]:
for i in range (1,5):
    for j in range (1,5):
        h = g2_heisto(astamps, achannel,bstamps, bchannel, amd=i, bmd=j,cwindow=500,search_steps=20,search_direction=-1)
        print ("detectors", detectors[i],detectors[j],"\tshiftsteps",h.index(max(h)),"\tcoincount",max(h))

detectors H H 	shiftsteps 9 	coincount 2753
detectors H V 	shiftsteps 25 	coincount 54
detectors H A 	shiftsteps 15 	coincount 1093
detectors H D 	shiftsteps 8 	coincount 1750
detectors V H 	shiftsteps 10 	coincount 53
detectors V V 	shiftsteps 11 	coincount 2462
detectors V A 	shiftsteps 16 	coincount 1280
detectors V D 	shiftsteps 10 	coincount 1156
detectors A H 	shiftsteps 10 	coincount 717
detectors A V 	shiftsteps 11 	coincount 751
detectors A A 	shiftsteps 16 	coincount 1524
detectors A D 	shiftsteps 9 	coincount 99
detectors D H 	shiftsteps 8 	coincount 650
detectors D V 	shiftsteps 8 	coincount 801
detectors D A 	shiftsteps 13 	coincount 66
detectors D D 	shiftsteps 7 	coincount 1521


In [135]:
def correct_timestamp(stamps,detectors,sH=0,sV=0,sA=0,sD=0):
    cstamp=[]
    ts = 0
    for (s,d) in zip (stamps,detectors):
        ts = s
        if(d==H): 
            ts+=sH
        elif(d==V):
            ts+=sV
        elif(d==A):
            ts+=sA
        elif(d==D):
            ts+=sD      
        cstamp.append(ts)
    return cstamp

In [152]:
#correct alice HV delay missmatch add 500ps to alice's V 
astamps_corrected = correct_timestamp(astamps,achannel,sH=2*500,sV=4*500,sA=6*500+3*500,sD=3*500+3*500)

In [153]:
#h = g2_heisto(astamps_corrected, achannel,bstamps, bchannel, amd=H, bmd=H,cwindow=500,search_steps=30,search_direction=-1)
#print (h)
#h.index(max(h))

In [154]:
#correct Bob
# add 7 To Bob's A basis
bstamps_corrected = correct_timestamp(bstamps,bchannel,sH=0,sV=0,sA=0,sD=6*500)

In [155]:
for i in range (1,5):
    for j in range (1,5):
        h = g2_heisto(astamps_corrected, achannel,bstamps_corrected, bchannel, amd=i, bmd=j,cwindow=500,search_steps=20,search_direction=-1)
        print ("detectors", detectors[i],detectors[j],"\tshiftsteps",h.index(max(h)),"\tcoincount",max(h))

detectors H H 	shiftsteps 7 	coincount 2753
detectors H V 	shiftsteps 7 	coincount 53
detectors H A 	shiftsteps 13 	coincount 1093
detectors H D 	shiftsteps 12 	coincount 1750
detectors V H 	shiftsteps 6 	coincount 53
detectors V V 	shiftsteps 7 	coincount 2462
detectors V A 	shiftsteps 12 	coincount 1280
detectors V D 	shiftsteps 12 	coincount 1156
detectors A H 	shiftsteps 1 	coincount 717
detectors A V 	shiftsteps 2 	coincount 751
detectors A A 	shiftsteps 7 	coincount 1524
detectors A D 	shiftsteps 6 	coincount 99
detectors D H 	shiftsteps 2 	coincount 650
detectors D V 	shiftsteps 2 	coincount 801
detectors D A 	shiftsteps 7 	coincount 66
detectors D D 	shiftsteps 7 	coincount 1521


In [156]:
#h = g2_heisto(astamps_corrected, achannel,bstamps_corrected, bchannel, amd=A, bmd=H,cwindow=500,search_steps=20,search_direction=-1)
#print (h)
#h.index(max(h))

In [157]:
def match_coincidence(at,ad,bt,bd,cwindow,shift):
    ia = 0
    ib = 0
    acoin_indexes = []
    bcoin_indexes = []
    while (ia<len(at) and ib <len(bt)):
        
        diff = bt[ib]- at[ia] + shift

        if (abs(diff)<=cwindow):
            #coin_count +=1
            acoin_indexes.append(ia)
            bcoin_indexes.append(ib)
            ia+=1
            ib+=1
            #continue
        elif (diff>cwindow):
            ia+=1
            #continue
        elif( diff <-cwindow):
            ib+=1
            #continue
        else:
            print ("ERROR! I should not be here")
            break
    return acoin_indexes,bcoin_indexes
    

In [191]:
ac_indexes,bc_indexes = match_coincidence(astamps_corrected,achannel,bstamps_corrected,bchannel,3000,-7*500)

In [192]:
len(ac_indexes)

20061

In [193]:
len(ac_indexes)/ len(astamps)

0.22300158961304595

In [181]:
ac_indexes[:10]

[1, 5, 8, 9, 11, 16, 20, 21, 23, 24]

In [182]:
bc_indexes[:10]

[1, 2, 10, 11, 12, 18, 24, 25, 30, 32]

In [183]:
print(astamps_corrected[51],bstamps_corrected[76], bstamps_corrected[76] - astamps_corrected[51]-9*500)

1086407877784 1086410543788 2661504


In [184]:
basis_count = 0
error_count = 0
for (a,b) in zip(ac_indexes,bc_indexes):
    da = achannel[a]
    db = bchannel[b]
    if (da >= 3 and db >= 3):
        #basis HV
        #print(da,db)
        basis_count+=1
        if (da!=db):
            error_count+=1
    if( da<3 and db <3):
        #print (da,db)
        if(da!=db):
            error_count+=1
        basis_count+=1

In [171]:
basis_count

10119

In [172]:
error_count

478

In [173]:
QBER = error_count / basis_count
QBER*100

4.723786935467931

In [174]:
astamps_corrected[0:10]

[1086398753803,
 1086398896346,
 1086398986766,
 1086399118932,
 1086399160249,
 1086399265520,
 1086399494780,
 1086399692867,
 1086400242735,
 1086400303477]

In [175]:
bstamps_corrected[0:10]

[1086398875555,
 1086398902493,
 1086399266452,
 1086399304476,
 1086399336997,
 1086399344556,
 1086399650658,
 1086399826464,
 1086400121460,
 1086400189646]

In [24]:
len (ac_indexes)

3338337

In [26]:
duration = astamps[-1]-astamps[0]
duration / (10**12)

9.966141765334

In [28]:
len(astamps)/10

3434290.3

Singles rate 3.4 Million/s
coincidence rate 333 K/s 
QBER = 3.7 %
coincidence window 500 ps

for coincidence window of 1 ns QBER is around 4.2 %
